In [ ]:
list_authors = df["author"].value_counts().index.tolist()

def make_histplot(stat, x_label, y_label, ax):
    # Draw the histogram and fit a density plot.
    sns.histplot(x=df.loc[df["author"]==author].num_of_tokens, kde=True, ax=ax)
    x=df.loc[df["author"]==author].num_of_tokens
    # get the y-coordinates of the points of the density curve.
    dens_list = ax.get_lines()[0].get_data()[1]

    # find the maximum y-coordinates of the density curve.
    max_dens_index = dens_list.argmax()

    # find the mode of the density plot.
    mode_x = ax.get_lines()[0].get_data()[0][max_dens_index]
    
    # draw a vertical line at the mode of the histogram.
    ax.axvline(mode_x, label='mode', color='blue', linestyle='dashed', linewidth=1.5)
    
    
    # draw a vertical line at the mean of the histogram.
    ax.axvline(x.mean(), label='mean',linestyle='dashed', color='red', linewidth=1.5)
    

    # draw a vertical line at the median of the histogram.
    ax.axvline(x.median(), label='median',linestyle='dashed', color='green', linewidth=1.5)
    ax.text(x.median(), 0.16, '{:.4f}'.format(x.median()))

    ax.legend()
    # Plot formatting
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)

num_subplots = len(list_authors)
ncols = 4

nrows = 9
fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols * 6, nrows * 5))
#colors = plt.cm.tab10.colors
for ax, author in zip(np.ravel(axes), list_authors):
    make_histplot(author, author, 'number of appearance', ax)
#for ax in np.ravel(axes)[num_subplots:]:  # remove possible empty subplots at the end
    #ax.remove()
plt.show()

In [ ]:
sns.displot(df,x="num_of_tokens",hue="school",kind="kde",fill=True, height=8.27, aspect=21.7/8.27)

In [ ]:
# plot sentence length split by school
plt.figure(figsize=(16,5))
sns.violinplot(x='school', y='num_of_tokens', data=df)
plt.title('num_of_tokens - By School')
plt.grid()

In [ ]:
def create_notion(tokenized_txt):
    notions=[]
    a=0
    tokenized_txt=ast.literal_eval(tokenized_txt)
    for i in tokenized_txt:
        if i not in common_words:
            notions.append(i)
            a=a+1
    return(a,notions)
df["notions"]=df["tokenized_txt"].apply(create_notion)
df["num_of_notions"]=df["notions"].apply(lambda x :x[0])
df["notions_list"]=df["notions"].apply(lambda x :x[1])

In [ ]:
my_cmap = plt.get_cmap("viridis")
rescale = lambda y: (y - np.min(y)) / (np.max(y) - np.min(y))

Uncommonness=(df.groupby("author").sum()["num_of_notions"]/df.groupby("author").sum()["num_of_tokens"]).sort_values()
fig,ax=plt.subplots(figsize=(30,10))
plt.bar(Uncommonness.index,Uncommonness.values,color=my_cmap(rescale(Uncommonness.values)))
ax.tick_params(labelsize=15)
for item in ax.xaxis.get_ticklabels():
    item.set_rotation(40)
plt.xlabel('Philosopher', fontsize=18)
plt.ylabel('Uncommon word density', fontsize=16)
plt.title("Difficultness of reading according to use of uncommon words",fontsize=30)

In [ ]:
t1 = time.time()
stopwords=set(common_words)
schools = df["school"].value_counts().index.tolist()
for sc in schools:
    df_temp = df[df.school==sc]
    
    print('School = ', sc.upper(), ':')
    
    # render wordcloud
    text = " ".join(txt for txt in df_temp.sentence_lowered)
    wordcloud = WordCloud(stopwords=stopwords, max_font_size=50, max_words=500,
                          width = 600, height = 400,
                          background_color="white").generate(text)
    plt.figure(figsize=(12,8))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()
t2 = time.time()
print('Elapsed time: ', np.round(t2-t1,2))

In [ ]:
import nltk
nltk.download('vader_lexicon')
def SentimentAnlysis(sentence):
    sentAnalyzer = SentimentIntensityAnalyzer() 
    sentDict = sentAnalyzer.polarity_scores(sentence)
    
    if sentDict['compound'] >= 0.05:
        return "positive"
    elif sentDict['compound'] <= -0.05 :
        return "negative"
    else:
        return "neutral"


# Our Main Function
def Analyzer(dataFrame, author):
    df = dataFrame [dataFrame['author'] == author] 
        
    #making a Corpus and finding sentiments
    corpus = ''
    numPostives = 0
    numNegatives = 0
    numNeutrals = 0
    
    for mem in df['sentence_lowered']:
        corpus += mem
    
    for i in range (len(df)):
        sent = (SentimentAnlysis(df['sentence_lowered'].iloc[i]))
        if sent == "positive":
            numPostives += 1
        elif sent == "negative":
            numNegatives += 1
        else:
            numNeutrals += 1
    
    plt.figure(figsize = (7, 7))
    plt.pie([numPostives, numNegatives, numNeutrals], labels = ['positives', 'negatives', 'neutrals'], autopct='%1.2f%%')
    plt.title('Sentiment Analysis for  Philosopher: ' + author)